# Observation / Action Specifiction (Clean Implementation)

TODO:
- [x] Observation Spec `replay_df`
   - [x] Champs
   - [x] Minions
   - [x] Turrets
   - [x] Monsters
   - [x] Missiles
- [ ] Action Spec `replay_df`:
   - [x] Movement
   - [-] Recall (Code available)
   - [x] Spell
     - [x] Q
     - [x] W
     - [x] E (Get direction in digits)
     - [x] D (Flash)
     - [x] F (Alt)
   - [ ] Auto Attack

## Populate Game Object Dataframes

In [111]:
import os
import pandas as pd
import numpy as np

db_replays_dir = "/Users/joe/Downloads/DB"
db_replays = os.listdir(db_replays_dir)

In [112]:
import sqlite3
import pandas as pd
import os
db_replay = os.path.join(db_replays_dir, db_replays[0])
con = sqlite3.connect(db_replay)

In [113]:
AUTO_ATTACK_TARGETS = ["CHAMPS", "TURRETS", "MINIONS", "MISSILES", "MONSTERS", "OTHER"]

In [114]:
GAME_OBJECT_LIST = ["champs", "turrets", "minions", "missiles", "monsters"]

In [115]:
MAX_OBJS = [10, 30, 30, 30, 30]

In [116]:
df_s = {
    obj:pd.read_sql(f"SELECT * FROM {obj};", con) for obj in GAME_OBJECT_LIST}

## Dataframe Preprocessing

### Data Cleaning for all Game Object Dataframes

In [117]:
for obj in GAME_OBJECT_LIST:
    df_s[obj] = df_s[obj].drop_duplicates(
        subset=["time", "name"])
    df_s[obj] = df_s[obj][
        df_s[obj]["time"] > 15]
    df_s[obj] = df_s[obj].drop("game_id", axis=1)

### Data Normalisation for Champs DF

In [118]:
df_s["champs"].loc[df_s["champs"]['q_cd'] < 0, 'q_cd'] = 0
df_s["champs"].loc[df_s["champs"]['w_cd'] < 0, 'w_cd'] = 0
df_s["champs"].loc[df_s["champs"]['e_cd'] < 0, 'e_cd'] = 0
df_s["champs"].loc[df_s["champs"]['r_cd'] < 0, 'r_cd'] = 0
df_s["champs"].loc[df_s["champs"]['d_cd'] < 0, 'd_cd'] = 0
df_s["champs"].loc[df_s["champs"]['f_cd'] < 0, 'f_cd'] = 0

## Dataframe Feature Engineering

In [119]:
player_df = df_s["champs"][df_s["champs"]["name"] == "Ezreal"]

In [120]:
player_df

,time,name,hp,max_hp,mana,max_mana,armor,mr,ad,ap,...,w_name,w_cd,e_name,e_cd,r_name,r_cd,d_name,d_cd,f_name,f_cd
503,15.034752,Ezreal,700.00,700.00,375.00000,375.00,36.0000,30.0000,77.4000,0.0,...,EzrealW,0.0,EzrealE,0.000000,EzrealR,0.0,SummonerHeal,0.965248,SummonerFlash,0.965248
513,15.303469,Ezreal,700.00,700.00,375.00000,375.00,36.0000,30.0000,77.4000,0.0,...,EzrealW,0.0,EzrealE,0.000000,EzrealR,0.0,SummonerHeal,0.696531,SummonerFlash,0.696531
523,15.581064,Ezreal,700.00,700.00,375.00000,375.00,36.0000,30.0000,77.4000,0.0,...,EzrealW,0.0,EzrealE,0.000000,EzrealR,0.0,SummonerHeal,0.418936,SummonerFlash,0.418936
533,15.849633,Ezreal,700.00,700.00,375.00000,375.00,36.0000,30.0000,77.4000,0.0,...,EzrealW,0.0,EzrealE,0.000000,EzrealR,0.0,SummonerHeal,0.150367,SummonerFlash,0.150367
543,16.139160,Ezreal,700.00,700.00,375.00000,375.00,36.0000,30.0000,77.4000,0.0,...,EzrealW,0.0,EzrealE,0.000000,EzrealR,0.0,SummonerHeal,0.000000,SummonerFlash,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6643,178.674740,Ezreal,850.45,850.45,327.76978,478.25,42.9325,31.9175,81.0875,0.0,...,EzrealW,0.0,EzrealE,13.740845,EzrealR,0.0,SummonerHeal,0.000000,SummonerFlash,0.000000
6653,178.970980,Ezreal,850.45,850.45,328.76727,478.25,42.9325,31.9175,81.0875,0.0,...,EzrealW,0.0,EzrealE,13.444611,EzrealR,0.0,SummonerHeal,0.000000,SummonerFlash,0.000000
6663,179.318790,Ezreal,850.45,850.45,328.76727,478.25,42.9325,31.9175,81.0875,0.0,...,EzrealW,0.0,EzrealE,13.096802,EzrealR,0.0,SummonerHeal,0.000000,SummonerFlash,0.000000
6683,179.636140,Ezreal,850.45,850.45,329.76477,478.25,42.9325,31.9175,81.0875,0.0,...,EzrealW,0.0,EzrealE,12.779449,EzrealR,0.0,SummonerHeal,0.000000,SummonerFlash,0.000000


## Observation Spec

### Init `replay_df`

In [121]:
replay_df = pd.DataFrame()
times = df_s["champs"].drop_duplicates(subset=['time'])["time"]
replay_df["time"] = times

### Add Distance Between Local Player and All Game Objects

In [122]:
player_df = df_s["champs"][df_s["champs"]["name"] == "Ezreal"]

In [123]:
def add_distances(original_df):
    # Step 1: Filter out Player's data
    player_df_data = player_df[['time', 'pos_x', 'pos_z']]

    # Step 2: Merge with the original DataFrame on 'time'
    merged_df = original_df.merge(player_df_data, on='time', suffixes=('', '_player'))

    # Step 3: Calculate Euclidean distance
    merged_df['distance_from_player_x'] = abs(
        merged_df["pos_x_player"] - merged_df["pos_x"])
    merged_df['distance_from_player_z'] = abs(
        merged_df["pos_z_player"] - merged_df["pos_z"])
    merged_df['distance_from_player'] = np.sqrt(
        (merged_df['pos_x'] - merged_df['pos_x_player'])**2 +
        (merged_df['pos_z'] - merged_df['pos_z_player'])**2)
    merged_df = merged_df.drop(columns=["pos_x_player", "pos_z_player"])

    # Filter out the rows where the champion is Ezreal, as we don't need the distance of Ezreal to himself
    return merged_df

for obj in GAME_OBJECT_LIST:
    df_s[obj] = add_distances(df_s[obj])

### Flatten Observations

In [124]:
testdf_s = {}

#### Flatten Each Dataframe

In [125]:
times = df_s["champs"]["time"].unique()

def flatten_obs(df, max_objs, times):
    # print(df.columns)
    new_cols = [f"{c}_{idx}"
                for idx in range(max_objs)
                for c in df.columns[1:]]
    new_cols = ["time"] + new_cols
    obs   = []
    for tm in times:
        cur = df[df["time"] == tm]
        vals = cur.values[:, 1:]
        vals = vals[:max_objs, :]

        if cur.shape[0] < max_objs:
            padding_val = max_objs - vals.shape[0]
            padding = np.zeros((padding_val, vals.shape[1]))
            vals = np.vstack((vals, padding))
    
        # Flatten
        new_vals = np.hstack(vals)

        # Append
        obs.append(new_vals)

    # Combine
    obs   = np.vstack(obs)
    times = np.expand_dims(np.array(times), axis=1)
    obs   = np.hstack((times, obs))
    # print(obs.shape)
    obs_df = pd.DataFrame(data=obs, columns=new_cols)

    return obs_df

for obj, max_objs in zip(GAME_OBJECT_LIST, MAX_OBJS):
    # print("OBJ, MAX:", obj, max_objs)
    testdf_s[obj] = flatten_obs(df_s[obj], max_objs, times)

#### Combine All Flattened Dataframes

In [126]:
times_unsqueeze = np.expand_dims(times, 1)
replay_df_vals = [df.iloc[:, 1:] for df in testdf_s.values()]
replay_df_vals = np.hstack(replay_df_vals)
print(times_unsqueeze.shape, replay_df_vals.shape)
replay_df_vals = np.hstack((times_unsqueeze, replay_df_vals))

(574, 1) (574, 2340)


In [127]:
replay_df_vals.shape

(574, 2341)

In [128]:
def flatten_list(lst):
    return [x for xs in lst for x in xs]

replay_df_cols = [list(testdf_s[k].columns[1:].values) for k in testdf_s.keys()]
replay_df_cols = flatten_list(replay_df_cols)
replay_df_cols = ["time"] + replay_df_cols

In [129]:
replay_df = pd.DataFrame(
    data=replay_df_vals,
    columns=replay_df_cols)
replay_df

,time,name_0,hp_0,max_hp_0,mana_0,max_mana_0,armor_0,mr_0,ad_0,ap_0,...,ap_29,level_29,atk_range_29,visible_29,team_29,pos_x_29,pos_z_29,distance_from_player_x_29,distance_from_player_z_29,distance_from_player_29
0,15.034752,Akali,570.0,570.0,200.0,200.0,35.0,37.0,67.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.303469,Akali,570.0,570.0,200.0,200.0,35.0,37.0,67.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15.581064,Akali,570.0,570.0,200.0,200.0,35.0,37.0,67.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15.849633,Akali,570.0,570.0,200.0,200.0,35.0,37.0,67.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16.13916,Akali,570.0,570.0,200.0,200.0,35.0,37.0,67.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,178.67474,Akali,574.6728,771.68,78.277725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
570,178.97098,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
571,179.31879,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
572,179.63614,Akali,577.9464,771.68,89.777725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Action Spec

In [130]:
player_df

,time,name,hp,max_hp,mana,max_mana,armor,mr,ad,ap,...,w_name,w_cd,e_name,e_cd,r_name,r_cd,d_name,d_cd,f_name,f_cd
503,15.034752,Ezreal,700.00,700.00,375.00000,375.00,36.0000,30.0000,77.4000,0.0,...,EzrealW,0.0,EzrealE,0.000000,EzrealR,0.0,SummonerHeal,0.965248,SummonerFlash,0.965248
513,15.303469,Ezreal,700.00,700.00,375.00000,375.00,36.0000,30.0000,77.4000,0.0,...,EzrealW,0.0,EzrealE,0.000000,EzrealR,0.0,SummonerHeal,0.696531,SummonerFlash,0.696531
523,15.581064,Ezreal,700.00,700.00,375.00000,375.00,36.0000,30.0000,77.4000,0.0,...,EzrealW,0.0,EzrealE,0.000000,EzrealR,0.0,SummonerHeal,0.418936,SummonerFlash,0.418936
533,15.849633,Ezreal,700.00,700.00,375.00000,375.00,36.0000,30.0000,77.4000,0.0,...,EzrealW,0.0,EzrealE,0.000000,EzrealR,0.0,SummonerHeal,0.150367,SummonerFlash,0.150367
543,16.139160,Ezreal,700.00,700.00,375.00000,375.00,36.0000,30.0000,77.4000,0.0,...,EzrealW,0.0,EzrealE,0.000000,EzrealR,0.0,SummonerHeal,0.000000,SummonerFlash,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6643,178.674740,Ezreal,850.45,850.45,327.76978,478.25,42.9325,31.9175,81.0875,0.0,...,EzrealW,0.0,EzrealE,13.740845,EzrealR,0.0,SummonerHeal,0.000000,SummonerFlash,0.000000
6653,178.970980,Ezreal,850.45,850.45,328.76727,478.25,42.9325,31.9175,81.0875,0.0,...,EzrealW,0.0,EzrealE,13.444611,EzrealR,0.0,SummonerHeal,0.000000,SummonerFlash,0.000000
6663,179.318790,Ezreal,850.45,850.45,328.76727,478.25,42.9325,31.9175,81.0875,0.0,...,EzrealW,0.0,EzrealE,13.096802,EzrealR,0.0,SummonerHeal,0.000000,SummonerFlash,0.000000
6683,179.636140,Ezreal,850.45,850.45,329.76477,478.25,42.9325,31.9175,81.0875,0.0,...,EzrealW,0.0,EzrealE,12.779449,EzrealR,0.0,SummonerHeal,0.000000,SummonerFlash,0.000000


### Movement

In [131]:
import math

player_cur_pos  = player_df[["time", "pos_x", "pos_z"]]
player_next_pos = player_df[["time", "pos_x", "pos_z"]].shift(1)
player_next_pos = player_next_pos.fillna(0)
player_x_delta  = player_next_pos["pos_x"] - player_cur_pos["pos_x"]
player_z_delta  = player_next_pos["pos_z"] - player_cur_pos["pos_z"]
player_x_delta.iloc[0] = 0
player_z_delta.iloc[0] = 0

player_x_delta_digit = (player_x_delta / 100).round().clip(-4, +4)
player_z_delta_digit = (player_z_delta / 100).round().clip(-4, +4)

player_df["player_x_delta"]       = player_x_delta
player_df["player_z_delta"]       = player_z_delta
player_df["player_delta"]         = np.sqrt(player_x_delta ** 2 + player_z_delta ** 2)
player_df["player_x_delta_digit"] = player_x_delta_digit
player_df["player_z_delta_digit"] = player_z_delta_digit

In [132]:
player_movement_df_final = \
    player_df[["time", "player_x_delta_digit", "player_z_delta_digit"]]
player_movement_df_final

,time,player_x_delta_digit,player_z_delta_digit
503,15.034752,0.0,0.0
513,15.303469,1.0,0.0
523,15.581064,1.0,-0.0
533,15.849633,-0.0,-1.0
543,16.139160,-1.0,-1.0
...,...,...,...
6643,178.674740,-0.0,-0.0
6653,178.970980,0.0,0.0
6663,179.318790,-0.0,1.0
6683,179.636140,-1.0,-1.0


In [133]:
replay_df = pd.merge(replay_df, player_movement_df_final, on="time")
replay_df

,time,name_0,hp_0,max_hp_0,mana_0,max_mana_0,armor_0,mr_0,ad_0,ap_0,...,atk_range_29,visible_29,team_29,pos_x_29,pos_z_29,distance_from_player_x_29,distance_from_player_z_29,distance_from_player_29,player_x_delta_digit,player_z_delta_digit
0,15.034752,Akali,570.0,570.0,200.0,200.0,35.0,37.0,67.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.303469,Akali,570.0,570.0,200.0,200.0,35.0,37.0,67.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,15.581064,Akali,570.0,570.0,200.0,200.0,35.0,37.0,67.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.0
3,15.849633,Akali,570.0,570.0,200.0,200.0,35.0,37.0,67.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0
4,16.13916,Akali,570.0,570.0,200.0,200.0,35.0,37.0,67.4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,178.67474,Akali,574.6728,771.68,78.277725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0
570,178.97098,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
571,179.31879,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,1.0
572,179.63614,Akali,577.9464,771.68,89.777725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0


### Recall (Not Using this Yet)

In [134]:
still_frames_matches = ((player_df["player_x_delta_digit"] == 0) & \
(player_df["player_z_delta_digit"] == 0))
still_frames_idx = still_frames_matches[still_frames_matches].index
still_frames = player_df.loc[still_frames_idx]

still_frames["time_diff"] = still_frames["time"].diff().fillna(0)
still_frames['consecutive'] = still_frames['time_diff'] < 1

still_frames['group'] = (~still_frames['consecutive']).cumsum()

# Get the start and end times for each group
group_start_end_times = still_frames.groupby('group')['time'].agg(['min', 'max'])

# Calculate the duration of each group
group_start_end_times['duration'] = group_start_end_times['max'] - group_start_end_times['min']

# Find groups where the duration is at least 8 seconds
long_groups = group_start_end_times[group_start_end_times['duration'] >= 8].index

# Extract all rows belonging to these groups
long_duration_groups = still_frames[still_frames['group'].isin(long_groups)]
long_duration_groups

,time,name,hp,max_hp,mana,max_mana,armor,mr,ad,ap,...,f_name,f_cd,player_x_delta,player_z_delta,player_delta,player_x_delta_digit,player_z_delta_digit,time_diff,consecutive,group


### Spells

In [135]:
def swap_columns(df, col1, col2):
    """
    Swap two columns in a pandas DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame in which to swap columns.
    col1 (str): The name of the first column to swap.
    col2 (str): The name of the second column to swap.
    """
    temp = df[col1].copy()
    df[col1] = df[col2]
    df[col2] = temp
    return df

In [136]:
d_name = player_df["d_name"]
if d_name.iloc[0] != "SummonerFlash":
    swap_columns(player_df, "d_name", "f_name")
    swap_columns(player_df, "d_cd", "f_cd")

In [137]:
d_name = player_df["d_name"]
d_name

503     SummonerFlash
513     SummonerFlash
523     SummonerFlash
533     SummonerFlash
543     SummonerFlash
            ...      
6643    SummonerFlash
6653    SummonerFlash
6663    SummonerFlash
6683    SummonerFlash
6693    SummonerFlash
Name: d_name, Length: 574, dtype: object

In [138]:
spell_casts = player_df[["time", "q_cd", "w_cd", "e_cd", "r_cd", "d_cd", "f_cd"]]

#### Q

In [139]:
# Get every time q was cast (this will be 100% acc)
qcd_s_cur  = spell_casts["q_cd"]
qcd_s_prev = spell_casts["q_cd"].shift(+1).fillna(0)
qcd_s_diff = qcd_s_cur - qcd_s_prev
q_cast     = (qcd_s_diff > 4) & (qcd_s_cur > 0)
# spell_casts_full = spell_casts.merge(q_cast, on=["time"])
print("Number of ezreal q casts:", q_cast.sum())

# Look through ezreal missiles to find the ezreal q's
# (This will be less than 100% acc because of timing issue with scraping
# missiles)
ezreal_q_df = pd.read_sql(
    "SELECT time, start_pos_x, start_pos_z, end_pos_x, end_pos_z FROM missiles WHERE spell_name = 'EzrealQ';",
    con)
ezreal_q_df = ezreal_q_df.drop_duplicates(
    subset=['start_pos_x', 'start_pos_z'])
print(ezreal_q_df)

# Get q spell aim as digital
ezreal_q_start_pos  = ezreal_q_df[["time", "start_pos_x", "start_pos_z"]]
ezreal_q_end_pos = ezreal_q_df[["time", "end_pos_x", "end_pos_z"]]
ezreal_q_end_pos = ezreal_q_end_pos.fillna(0)
ezreal_q_x_delta  = ezreal_q_end_pos["end_pos_x"] - ezreal_q_start_pos["start_pos_x"]
ezreal_q_z_delta  = ezreal_q_end_pos["end_pos_z"] - ezreal_q_start_pos["start_pos_z"]

ezreal_q_x_delta_digit = (ezreal_q_x_delta / 100).round().clip(-4, +4)
ezreal_q_z_delta_digit = (ezreal_q_z_delta / 100).round().clip(-4, +4)

ezreal_q_df["ezreal_q_x_delta"] = ezreal_q_x_delta
ezreal_q_df["ezreal_q_z_delta"] = ezreal_q_z_delta
ezreal_q_df["ezreal_q_x_delta_digit"] = ezreal_q_x_delta_digit
ezreal_q_df["ezreal_q_z_delta_digit"] = ezreal_q_z_delta_digit

Number of ezreal q casts: 9
          time  start_pos_x  start_pos_z  end_pos_x  end_pos_z
0   117.788864    11437.131    1661.6350  12563.790  2071.8050
2   127.895790    10879.960    1182.8914  11011.325  2374.6733
5   138.633450    11398.467    1294.7552  12315.749  2066.8910
8   152.625460    10730.425    1034.5089  11340.131  2066.9136
10  172.179610    10034.534    1048.1306  10985.169  1778.8120
15  178.416630    10251.595     886.4006  11126.689  1706.0410


In [140]:
ezreal_q_df_final = \
    ezreal_q_df[["time", "ezreal_q_x_delta_digit", "ezreal_q_z_delta_digit"]]
ezreal_q_df_final["using_q"] = True
ezreal_q_df_final

/var/folders/zb/zb7f652n559gx4c68jjjxzfh0000gn/T/ipykernel_1974/4283128283.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ezreal_q_df_final["using_q"] = True


,time,ezreal_q_x_delta_digit,ezreal_q_z_delta_digit,using_q
0,117.788864,4.0,4.0,True
2,127.895790,1.0,4.0,True
5,138.633450,4.0,4.0,True
8,152.625460,4.0,4.0,True
10,172.179610,4.0,4.0,True
15,178.416630,4.0,4.0,True


In [141]:
replay_df = pd.merge(replay_df, ezreal_q_df_final, on="time", how="left")

In [142]:
replay_df["using_q"] = replay_df["using_q"].fillna(False)
replay_df = replay_df.fillna(0)
replay_df

,time,name_0,hp_0,max_hp_0,mana_0,max_mana_0,armor_0,mr_0,ad_0,ap_0,...,pos_x_29,pos_z_29,distance_from_player_x_29,distance_from_player_z_29,distance_from_player_29,player_x_delta_digit,player_z_delta_digit,ezreal_q_x_delta_digit,ezreal_q_z_delta_digit,using_q
0,15.034752,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
1,15.303469,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,False
2,15.581064,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,-0.0,0.0,0.0,False
3,15.849633,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.0,-1.0,0.0,0.0,False
4,16.139160,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,178.674740,Akali,574.6728,771.68,78.277725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,False
570,178.970980,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
571,179.318790,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.0,1.0,0.0,0.0,False
572,179.636140,Akali,577.9464,771.68,89.777725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,False


#### W (not verified)

In [143]:
# Get every time w was cast (this will be 100% acc)
wcd_s_cur  = spell_casts["w_cd"]
wcd_s_prev = spell_casts["w_cd"].shift(+1).fillna(0)
wcd_s_diff = wcd_s_cur - wcd_s_prev
w_cast     = (wcd_s_diff > 11) & (wcd_s_cur > 0)
# spell_casts_full = spell_casts.merge(w_cast, on=["time"])
print("Number of ezreal w casts:", w_cast.sum())

# Look through ezreal missiles to find the ezreal w's
# (This will be less than 100% acc because of timing issue with scraping
# missiles)
ezreal_w_df = pd.read_sql(
    "SELECT time, start_pos_x, start_pos_z, end_pos_x, end_pos_z FROM missiles WHERE spell_name = 'EzrealW';",
    con)
ezreal_w_df = ezreal_w_df.drop_duplicates(
    subset=['start_pos_x', 'start_pos_z'])
#ezreal_w_df

# Get w spell aim as digital
ezreal_w_start_pos  = ezreal_w_df[["time", "start_pos_x", "start_pos_z"]]
ezreal_w_end_pos = ezreal_w_df[["time", "end_pos_x", "end_pos_z"]]
ezreal_w_end_pos = ezreal_w_end_pos.fillna(0)
ezreal_w_x_delta  = ezreal_w_end_pos["end_pos_x"] - ezreal_w_start_pos["start_pos_x"]
ezreal_w_z_delta  = ezreal_w_end_pos["end_pos_z"] - ezreal_w_start_pos["start_pos_z"]

ezreal_w_x_delta_digit = (ezreal_w_x_delta / 100).round().clip(-4, +4)
ezreal_w_z_delta_digit = (ezreal_w_z_delta / 100).round().clip(-4, +4)

ezreal_w_df["ezreal_w_x_delta"] = ezreal_w_x_delta
ezreal_w_df["ezreal_w_z_delta"] = ezreal_w_z_delta
ezreal_w_df["ezreal_w_x_delta_digit"] = ezreal_w_x_delta_digit
ezreal_w_df["ezreal_w_z_delta_digit"] = ezreal_w_z_delta_digit

Number of ezreal w casts: 0


In [144]:
ezreal_w_df_final = \
    ezreal_w_df[["time", "ezreal_w_x_delta_digit", "ezreal_w_z_delta_digit"]]
ezreal_w_df_final["using_w"] = True

In [145]:
replay_df = pd.merge(replay_df, ezreal_w_df_final, on="time", how="left")

In [146]:
replay_df

,time,name_0,hp_0,max_hp_0,mana_0,max_mana_0,armor_0,mr_0,ad_0,ap_0,...,distance_from_player_z_29,distance_from_player_29,player_x_delta_digit,player_z_delta_digit,ezreal_q_x_delta_digit,ezreal_q_z_delta_digit,using_q,ezreal_w_x_delta_digit,ezreal_w_z_delta_digit,using_w
0,15.034752,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,NaN,NaN,NaN
1,15.303469,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,False,NaN,NaN,NaN
2,15.581064,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,1.0,-0.0,0.0,0.0,False,NaN,NaN,NaN
3,15.849633,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,-0.0,-1.0,0.0,0.0,False,NaN,NaN,NaN
4,16.139160,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,-1.0,-1.0,0.0,0.0,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,178.674740,Akali,574.6728,771.68,78.277725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,-0.0,-0.0,0.0,0.0,False,NaN,NaN,NaN
570,178.970980,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,NaN,NaN,NaN
571,179.318790,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,-0.0,1.0,0.0,0.0,False,NaN,NaN,NaN
572,179.636140,Akali,577.9464,771.68,89.777725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,-1.0,-1.0,0.0,0.0,False,NaN,NaN,NaN


In [147]:

replay_df["using_w"] = replay_df["using_w"].fillna(False)
replay_df = replay_df.fillna(0)
replay_df

,time,name_0,hp_0,max_hp_0,mana_0,max_mana_0,armor_0,mr_0,ad_0,ap_0,...,distance_from_player_z_29,distance_from_player_29,player_x_delta_digit,player_z_delta_digit,ezreal_q_x_delta_digit,ezreal_q_z_delta_digit,using_q,ezreal_w_x_delta_digit,ezreal_w_z_delta_digit,using_w
0,15.034752,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,0,0,False
1,15.303469,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,False,0,0,False
2,15.581064,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,1.0,-0.0,0.0,0.0,False,0,0,False
3,15.849633,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,-0.0,-1.0,0.0,0.0,False,0,0,False
4,16.139160,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,-1.0,-1.0,0.0,0.0,False,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,178.674740,Akali,574.6728,771.68,78.277725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,-0.0,-0.0,0.0,0.0,False,0,0,False
570,178.970980,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,0,0,False
571,179.318790,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,-0.0,1.0,0.0,0.0,False,0,0,False
572,179.636140,Akali,577.9464,771.68,89.777725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,-1.0,-1.0,0.0,0.0,False,0,0,False


In [148]:
replay_df

,time,name_0,hp_0,max_hp_0,mana_0,max_mana_0,armor_0,mr_0,ad_0,ap_0,...,distance_from_player_z_29,distance_from_player_29,player_x_delta_digit,player_z_delta_digit,ezreal_q_x_delta_digit,ezreal_q_z_delta_digit,using_q,ezreal_w_x_delta_digit,ezreal_w_z_delta_digit,using_w
0,15.034752,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,0,0,False
1,15.303469,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,False,0,0,False
2,15.581064,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,1.0,-0.0,0.0,0.0,False,0,0,False
3,15.849633,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,-0.0,-1.0,0.0,0.0,False,0,0,False
4,16.139160,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,-1.0,-1.0,0.0,0.0,False,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,178.674740,Akali,574.6728,771.68,78.277725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,-0.0,-0.0,0.0,0.0,False,0,0,False
570,178.970980,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,False,0,0,False
571,179.318790,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,-0.0,1.0,0.0,0.0,False,0,0,False
572,179.636140,Akali,577.9464,771.68,89.777725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,-1.0,-1.0,0.0,0.0,False,0,0,False


#### E

In [149]:
# Get every time e was cast (this will be 100% acc)
ecd_s_cur  = spell_casts["e_cd"]
ecd_s_prev = spell_casts["e_cd"].shift(+1).fillna(0)
ecd_s_diff = ecd_s_cur - ecd_s_prev
e_cast     = (ecd_s_diff > 25) & (ecd_s_cur > 0)
print("Number of ezreal e casts:", e_cast.sum())
e_cast_idx_s = e_cast[e_cast].index

# create a dataframe with all of e cast action params
e_cast_vals = []
e_cast_cols = ["time", "ezreal_e_x_delta_digit", "ezreal_e_z_delta_digit"]
for e_cast_idx in e_cast_idx_s:
    # e_cast_idx = e_cast_idx_s[0]
    e_row = player_df.loc[e_cast_idx-50:e_cast_idx+50][["time", "player_delta"]]
    e_row_idx = e_row.idxmax()["player_delta"]
    tm, x, y = player_df.loc[e_row_idx][["time", "player_x_delta_digit", "player_z_delta_digit"]]
    e_cast_vals.append([tm, x, y])

ezreal_e_df_final = pd.DataFrame(data=e_cast_vals, columns=e_cast_cols)
ezreal_e_df_final["using_e"] = True
ezreal_e_df_final["using_e"] = ezreal_e_df_final["using_e"].fillna(False)
ezreal_e_df_final = ezreal_e_df_final.fillna(0)
ezreal_e_df_final

Number of ezreal e casts: 1


,time,ezreal_e_x_delta_digit,ezreal_e_z_delta_digit,using_e
0,168.84871,4.0,4.0,True


In [151]:
replay_df = pd.merge(replay_df, ezreal_e_df_final, on="time", how="left")

In [152]:
replay_df

,time,name_0,hp_0,max_hp_0,mana_0,max_mana_0,armor_0,mr_0,ad_0,ap_0,...,player_z_delta_digit,ezreal_q_x_delta_digit,ezreal_q_z_delta_digit,using_q,ezreal_w_x_delta_digit,ezreal_w_z_delta_digit,using_w,ezreal_e_x_delta_digit,ezreal_e_z_delta_digit,using_e
0,15.034752,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,0.0,False,0,0,False,NaN,NaN,NaN
1,15.303469,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,0.0,False,0,0,False,NaN,NaN,NaN
2,15.581064,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,-0.0,0.0,0.0,False,0,0,False,NaN,NaN,NaN
3,15.849633,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,-1.0,0.0,0.0,False,0,0,False,NaN,NaN,NaN
4,16.139160,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,-1.0,0.0,0.0,False,0,0,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,178.674740,Akali,574.6728,771.68,78.277725,300.0,38.384003,38.476,69.776,0.0,...,-0.0,0.0,0.0,False,0,0,False,NaN,NaN,NaN
570,178.970980,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,False,0,0,False,NaN,NaN,NaN
571,179.318790,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,1.0,0.0,0.0,False,0,0,False,NaN,NaN,NaN
572,179.636140,Akali,577.9464,771.68,89.777725,300.0,38.384003,38.476,69.776,0.0,...,-1.0,0.0,0.0,False,0,0,False,NaN,NaN,NaN


In [153]:
replay_df["using_e"] = replay_df["using_e"].fillna(False)
replay_df = replay_df.fillna(0)
replay_df

,time,name_0,hp_0,max_hp_0,mana_0,max_mana_0,armor_0,mr_0,ad_0,ap_0,...,player_z_delta_digit,ezreal_q_x_delta_digit,ezreal_q_z_delta_digit,using_q,ezreal_w_x_delta_digit,ezreal_w_z_delta_digit,using_w,ezreal_e_x_delta_digit,ezreal_e_z_delta_digit,using_e
0,15.034752,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,0.0,False,0,0,False,0.0,0.0,False
1,15.303469,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,0.0,False,0,0,False,0.0,0.0,False
2,15.581064,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,-0.0,0.0,0.0,False,0,0,False,0.0,0.0,False
3,15.849633,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,-1.0,0.0,0.0,False,0,0,False,0.0,0.0,False
4,16.139160,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,-1.0,0.0,0.0,False,0,0,False,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,178.674740,Akali,574.6728,771.68,78.277725,300.0,38.384003,38.476,69.776,0.0,...,-0.0,0.0,0.0,False,0,0,False,0.0,0.0,False
570,178.970980,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,0.0,False,0,0,False,0.0,0.0,False
571,179.318790,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,1.0,0.0,0.0,False,0,0,False,0.0,0.0,False
572,179.636140,Akali,577.9464,771.68,89.777725,300.0,38.384003,38.476,69.776,0.0,...,-1.0,0.0,0.0,False,0,0,False,0.0,0.0,False


#### Flash

In [154]:
# Get every time e was cast (this will be 100% acc)
dcd_s_cur  = spell_casts["d_cd"]
dcd_s_prev = spell_casts["d_cd"].shift(+1).fillna(0)
dcd_s_diff = dcd_s_cur - dcd_s_prev
d_cast     = (dcd_s_diff > 250) & (dcd_s_cur > 0)
d_cast_idx_s = d_cast[d_cast].index

d_cast_vals = []
d_cast_cols = ["time", "ezreal_d_x_delta_digit", "ezreal_d_z_delta_digit"]
for d_cast_idx in d_cast_idx_s:
    d_row = player_df.loc[d_cast_idx-50:d_cast_idx+50][["time", "player_delta"]]
    d_row_idx = d_row.idxmax()["player_delta"]
    tm, x, y = player_df.loc[d_row_idx][["time", "player_x_delta_digit", "player_z_delta_digit"]]
    d_cast_vals.append([tm, x, y])

ezreal_d_df_final = pd.DataFrame(data=d_cast_vals, columns=d_cast_cols)
ezreal_d_df_final["using_d"] = True
ezreal_d_df_final

,time,ezreal_d_x_delta_digit,ezreal_d_z_delta_digit,using_d


In [155]:
replay_df = pd.merge(replay_df, ezreal_d_df_final, on="time", how="left")

In [156]:
replay_df

,time,name_0,hp_0,max_hp_0,mana_0,max_mana_0,armor_0,mr_0,ad_0,ap_0,...,using_q,ezreal_w_x_delta_digit,ezreal_w_z_delta_digit,using_w,ezreal_e_x_delta_digit,ezreal_e_z_delta_digit,using_e,ezreal_d_x_delta_digit,ezreal_d_z_delta_digit,using_d
0,15.034752,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,False,0,0,False,0.0,0.0,False,NaN,NaN,NaN
1,15.303469,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,False,0,0,False,0.0,0.0,False,NaN,NaN,NaN
2,15.581064,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,False,0,0,False,0.0,0.0,False,NaN,NaN,NaN
3,15.849633,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,False,0,0,False,0.0,0.0,False,NaN,NaN,NaN
4,16.139160,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,False,0,0,False,0.0,0.0,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,178.674740,Akali,574.6728,771.68,78.277725,300.0,38.384003,38.476,69.776,0.0,...,False,0,0,False,0.0,0.0,False,NaN,NaN,NaN
570,178.970980,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,False,0,0,False,0.0,0.0,False,NaN,NaN,NaN
571,179.318790,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,False,0,0,False,0.0,0.0,False,NaN,NaN,NaN
572,179.636140,Akali,577.9464,771.68,89.777725,300.0,38.384003,38.476,69.776,0.0,...,False,0,0,False,0.0,0.0,False,NaN,NaN,NaN


In [157]:
replay_df["using_d"] = replay_df["using_d"].fillna(False)
replay_df = replay_df.fillna(0)
replay_df

,time,name_0,hp_0,max_hp_0,mana_0,max_mana_0,armor_0,mr_0,ad_0,ap_0,...,using_q,ezreal_w_x_delta_digit,ezreal_w_z_delta_digit,using_w,ezreal_e_x_delta_digit,ezreal_e_z_delta_digit,using_e,ezreal_d_x_delta_digit,ezreal_d_z_delta_digit,using_d
0,15.034752,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,False,0,0,False,0.0,0.0,False,0,0,False
1,15.303469,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,False,0,0,False,0.0,0.0,False,0,0,False
2,15.581064,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,False,0,0,False,0.0,0.0,False,0,0,False
3,15.849633,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,False,0,0,False,0.0,0.0,False,0,0,False
4,16.139160,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,False,0,0,False,0.0,0.0,False,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,178.674740,Akali,574.6728,771.68,78.277725,300.0,38.384003,38.476,69.776,0.0,...,False,0,0,False,0.0,0.0,False,0,0,False
570,178.970980,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,False,0,0,False,0.0,0.0,False,0,0,False
571,179.318790,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,False,0,0,False,0.0,0.0,False,0,0,False
572,179.636140,Akali,577.9464,771.68,89.777725,300.0,38.384003,38.476,69.776,0.0,...,False,0,0,False,0.0,0.0,False,0,0,False


#### Alt (Not using this yet)

In [158]:
fcd_s_cur  = spell_casts["f_cd"]
fcd_s_prev = spell_casts["f_cd"].shift(+1).fillna(0)
fcd_s_diff = fcd_s_cur - fcd_s_prev
f_cast     = (fcd_s_diff > 180) & (fcd_s_cur > 0)
f_cast_idx_s = f_cast[f_cast].index
print("Alt sum casts:", f_cast.sum())

Alt sum casts: 0


#### Auto Attack

In [159]:
"""
Determine auto attack target:
1. [x] Get all auto attack missile end positions
2. [x] Get all game object positions
3. [x] Match missile end locations with game object positions (within a tolerance)
4. [ ] Get IDX of target type
"""

# 1. Get all auto attack missile end positions
auto_attacks_df = pd.read_sql("SELECT * FROM missiles WHERE spell_name LIKE 'EzrealBasicAttack%';", con)
auto_attacks_df = auto_attacks_df.drop_duplicates(subset=["start_pos_x", "start_pos_z"])
auto_attacks_df

# 2. Get all game object positions
champ_pos_df    = df_s["champs"].drop_duplicates(subset=["pos_x", "pos_z"])
turrets_pos_df  = df_s["turrets"].drop_duplicates(subset=["pos_x", "pos_z"])
monsters_pos_df = df_s["monsters"].drop_duplicates(subset=["pos_x", "pos_z"])
minions_pos_df  = df_s["minions"].drop_duplicates(subset=["pos_x", "pos_z"])

# 3. Match missile end locations with game object positions (within a tolerance)
def find_aa_target(row):
    target_x, target_z = row["pos_x"], row["pos_z"]
    # For each champ pos, check against missile positions
    for _, aa in auto_attacks_df.iterrows():
        end_pos_x, end_pos_z = aa["end_pos_x"], aa["end_pos_z"]
        if target_x == end_pos_x and target_z == end_pos_z:
            return True
    return False

champ_found_aa    = champ_pos_df.apply(lambda r: find_aa_target(r), axis=1)
turrets_found_aa  = turrets_pos_df.apply(lambda r: find_aa_target(r), axis=1)
monsters_found_aa = monsters_pos_df.apply(lambda r: find_aa_target(r), axis=1)
minions_found_aa  = minions_pos_df.apply(lambda r: find_aa_target(r), axis=1)

print(auto_attacks_df.shape)
champ_found_aa.sum(), turrets_found_aa.sum(), monsters_found_aa.sum(), minions_found_aa.sum()

(7, 13)


(0, 0, 0, 5)

In [160]:
champ_autos    = champ_pos_df.loc[champ_found_aa[champ_found_aa].index][["time", "pos_x", "pos_z"]]
turrets_autos  = turrets_pos_df.loc[turrets_found_aa[turrets_found_aa].index][["time", "pos_x", "pos_z"]]
monsters_autos = monsters_pos_df.loc[monsters_found_aa[monsters_found_aa].index][["time", "pos_x", "pos_z"]]
minions_autos  = minions_pos_df.loc[minions_found_aa[minions_found_aa].index][["time", "pos_x", "pos_z"]]

champ_autos["auto_type"]    = "champ"
turrets_autos["auto_type"]  = "turret"
monsters_autos["auto_type"] = "monster"
minions_autos["auto_type"]  = "minion"

all_autos_cols = ["time", "auto_digit_x", "auto_digit_z", "target_type"]
all_autos_vals = np.vstack((champ_autos, turrets_autos, monsters_autos, minions_autos))
all_autos_df   = pd.DataFrame(data=all_autos_vals, columns=all_autos_cols)
all_autos_df["using_auto"] = True
all_autos_df

,time,auto_digit_x,auto_digit_z,target_type,using_auto
0,152.92659,11548.402,1578.0912,minion,True
1,162.4802,11307.99,1596.6964,minion,True
2,165.97435,11085.911,1589.094,minion,True
3,170.0425,10857.082,1177.6383,minion,True
4,170.77783,10626.0,1222.0,minion,True


In [161]:
replay_df = pd.merge(replay_df, all_autos_df, on="time", how="left")

In [162]:
replay_df

,time,name_0,hp_0,max_hp_0,mana_0,max_mana_0,armor_0,mr_0,ad_0,ap_0,...,ezreal_e_x_delta_digit,ezreal_e_z_delta_digit,using_e,ezreal_d_x_delta_digit,ezreal_d_z_delta_digit,using_d,auto_digit_x,auto_digit_z,target_type,using_auto
0,15.034752,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,False,0,0,False,NaN,NaN,NaN,NaN
1,15.303469,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,False,0,0,False,NaN,NaN,NaN,NaN
2,15.581064,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,False,0,0,False,NaN,NaN,NaN,NaN
3,15.849633,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,False,0,0,False,NaN,NaN,NaN,NaN
4,16.13916,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,False,0,0,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,178.67474,Akali,574.6728,771.68,78.277725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,False,0,0,False,NaN,NaN,NaN,NaN
570,178.97098,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,False,0,0,False,NaN,NaN,NaN,NaN
571,179.31879,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,False,0,0,False,NaN,NaN,NaN,NaN
572,179.63614,Akali,577.9464,771.68,89.777725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,False,0,0,False,NaN,NaN,NaN,NaN


In [163]:
replay_df["using_auto"] = replay_df["using_auto"].fillna(False)
replay_df = replay_df.fillna(0)
replay_df

,time,name_0,hp_0,max_hp_0,mana_0,max_mana_0,armor_0,mr_0,ad_0,ap_0,...,ezreal_e_x_delta_digit,ezreal_e_z_delta_digit,using_e,ezreal_d_x_delta_digit,ezreal_d_z_delta_digit,using_d,auto_digit_x,auto_digit_z,target_type,using_auto
0,15.034752,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,False,0,0,False,0.0,0.0,0,False
1,15.303469,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,False,0,0,False,0.0,0.0,0,False
2,15.581064,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,False,0,0,False,0.0,0.0,0,False
3,15.849633,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,False,0,0,False,0.0,0.0,0,False
4,16.139160,Akali,570.0000,570.00,200.000000,200.0,35.000000,37.000,67.400,0.0,...,0.0,0.0,False,0,0,False,0.0,0.0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,178.674740,Akali,574.6728,771.68,78.277725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,False,0,0,False,0.0,0.0,0,False
570,178.970980,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,False,0,0,False,0.0,0.0,0,False
571,179.318790,Akali,576.5816,771.68,84.027725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,False,0,0,False,0.0,0.0,0,False
572,179.636140,Akali,577.9464,771.68,89.777725,300.0,38.384003,38.476,69.776,0.0,...,0.0,0.0,False,0,0,False,0.0,0.0,0,False


## Get GameID

In [77]:
game_id = pd.read_sql("SELECT game_id FROM games;", con)
game_id

,game_id
0,2843393945
